- Do I need to predict edge label if yes how?


In [ ]:
# !pip install svgpathtools
#!pip uninstall torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu113.html

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import torch
from torch import tensor
print(f"Torch version: {torch.__version__}")
print(f"Cuda available: {torch.cuda.is_available()}")
import torch_geometric
print(f"Torch geometric version: {torch_geometric.__version__}")

/home/niyazi/mambaforge/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 1.13.0
Cuda available: True
Torch geometric version: 2.1.0


In [ ]:
from pathlib import Path
from svgpathtools import svg2paths
import cmath
from svgpathtools import Line,Arc
from svgpathtools import Path as svg_path
from svgpathtools import wsvg
from svgpathtools import disvg
import matplotlib.pyplot as plt
from math import pi
from math import tan, atan, sin ,degrees, radians
from os import walk
import torch
import torch_geometric
from torch_geometric.data import Dataset, Data
import os
from tqdm import tqdm
import networkx as nx
from torch_geometric.utils import to_networkx
from torch_geometric.utils import *
import matplotlib.pyplot as plt
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
from torch.nn import Linear, Dropout
from einops import rearrange, reduce
import timeit

# from torch_geometric.nn import GCNConv, GATv2Conv, GATConv
print(f"Torch version: {torch.__version__}")
print(f"Cuda available: {torch.cuda.is_available()}")
print(f"Torch geometric version: {torch_geometric.__version__}")

path = Path()

Torch version: 1.13.0
Cuda available: True
Torch geometric version: 2.1.0


In [ ]:
device ='cuda:0'

In [ ]:
torch.cuda.device_count()


2

In [ ]:
torch.cuda.current_device()

0

## Dataset

In [ ]:
class floorPlanCad(Dataset):
    
    def __init__(self, root,transform=None, pre_transform=None):
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data). 
        """
        #print(f'init called')
        
        self.root = root
        super().__init__(root, transform, pre_transform)
        # print(self.processed_dir)
        # print(self.raw_dir)
        # print(self.raw_paths)
        # print(type(Path(self.processed_dir)))
        
    @property
    def raw_file_names(self):
        #print(f'raw_file_names called')
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)  
        """
        self.filenames = next(walk(self.root/"raw"), (None, None, []))[2]
        return self.filenames

    @property
    def processed_file_names(self):
        #print(f'processed_file_names called')
        """ If these files are found in raw_dir, processing is skipped"""

        #return [f'd_{self.filenames.index(i)}.pt' for i in self.filenames]
        return [f'd_{i[:-4]}.pt' for i in self.filenames]
    

    def download(self):
        pass
 
    def process(self):  
        # print(f'process called')
        # edge_indices = [[1, 2], [2, 1]]
        # edge_torch = torch.tensor(edge_indices)
        # edge_torch = edge_torch.t().to(torch.long).view(2, -1)
        
        for i in tqdm(self.filenames): 
            
            #Node Features
            #------------------------------------------------------------------------
            current_drw = Path(self.raw_dir)/i
            drw_paths, drw_attributes = svg2paths(current_drw)
            node_features = []
            #node_features = torch.tensor([], dtype=torch.float)
            legal_paths=[]
            legal_attributes=[]
            node_class =[]
            removed_files = []
            # remove corrupt data
        
            for drw_attribute in drw_attributes:
                if ('semantic-id' and 'instance-id') in drw_attribute:
                    path_index = drw_attributes.index(drw_attribute)
                    drw_path = drw_paths[path_index]
                    node_class.append(int(drw_attribute['semantic-id']))
                    legal_paths.append(drw_path)
                    legal_attributes.append(drw_attribute)
            #print(len(legal_paths),len(legal_attributes))
            #some drawing has no legal paths        
            if not legal_paths:
                removed_files.append(i)
                (Path(self.raw_dir)/i).unlink()
                continue
            for legal_path in legal_paths:
                node_feature = self.node_features(legal_path)
                node_features += node_feature
                

            node_features = torch.tensor(node_features, dtype=torch.float)
            node_features = node_features.view(node_features.shape[0]//4,-1)
            node_class = torch.tensor(node_class)
            
            #y = torch.randint(0, 2, (node_features.shape[0],))
            
            #-------------------------------------------------------------------------
            #Edge Index
            edge_index =[]
            edge_feats =[]
            node_instance_matrix = []
            for legal_path in legal_paths:
                to_others=[]
                for other_legal_path in legal_paths:
                    if  (self.positional_offset(legal_path,other_legal_path)[1] < 4) or (self.start_to_end(legal_path,other_legal_path)):
                        # Edge Index
                        idx1 = legal_paths.index(legal_path)
                        idx2 = legal_paths.index(other_legal_path)
                        edge_index.append([idx1,idx2])
                        #------------------------------------------------------------------------- 
                        # Edge Features
                        
                        edge_feats.append(self.edge_features(legal_path,other_legal_path))
                        #this can be used for above loop too. Normally it creates self loops.
                        same = (idx1 == idx2)
                        
                        if (not same):                            
                            if (legal_attributes[idx1]['instance-id'] == legal_attributes[idx2]['instance-id']):                            
                                node_instance_matrix.append(1)
                            else:
                                node_instance_matrix.append(0)
                            
                        
                    #     #------------------------------------------------------------------------- 
                    # if (legal_path['instance-id']==other_legal_path['instance-id']):
                    #     print('same')
            # test value for extra property eventually. i reserved this for the second 'y' value 
            # for graph level prediction (instance prediction)
#            node_instance_matrix = []
#this is the case for that every node is included.            
#             for legal_attribute in legal_attributes:                
#                 for other_legal_attribute in legal_attributes:
#                     if  legal_attribute['instance-id'] == other_legal_attribute['instance-id']:
#                         print(legal_attribute['instance-id'],other_legal_attribute['instance-id'])
#                         node_instance_matrix.append(1)
#                     else:
#                         node_instance_matrix.append(0)
                        
                    
            
  
            edge_index = torch.tensor(edge_index)
            edge_index = edge_index.t().to(torch.long).view(2, -1)
            #print(edge_index.shape)
            edge_feats = torch.tensor(edge_feats, dtype=torch.float)
            #print(len(node_instance_matrix))
            node_instance_matrix = torch.tensor(node_instance_matrix, dtype=torch.float).unsqueeze(1)
            #print(node_instance_matrix.shape)
            node_instance_matrix = torch.abs(torch.cat((node_instance_matrix,node_instance_matrix-1),dim=1).to(torch.float))
            data = Data(x=node_features, 
                                edge_index = edge_index,
                                #edge_attr = torch.tensor([2,2], dtype=torch.float),
                                edge_attr = edge_feats,
                                y=node_class, yy=node_instance_matrix)
            
            data.edge_index,data.edge_attr = remove_self_loops(data.edge_index,data.edge_attr)
            
            torch.save(data,os.path.join(self.processed_dir,f'd_{i[:-4]}.pt'))
        print(removed_files)

    def path_to_seg(self,d_path):
        return d_path[0]

    def angle_line(self,d_path):
        angle_max=6.28
        seg = self.path_to_seg(d_path)
        delta = seg.point(1) - seg.point(0)
        polar = cmath.polar(delta)
        angle = polar[-1]
        if angle < 0 : angle += 2* pi
        #normalized angle
        return angle / angle_max

    def length(self,d_path):
        max_length = 66
        seg = self.path_to_seg(d_path)
        return seg.length() / max_length

    
    
    def path_type(self,d_path):
        prim_types= 'Line', 'Arc'
        seg = self.path_to_seg(d_path)
        one_hot=[]
        for d in prim_types:
            one_hot.append(d == seg.__class__.__name__)
        return one_hot


    def node_features(self,d_path):
        return [self.angle_line(d_path),self.length(d_path), self.path_type(d_path)[0],self.path_type(d_path)[1]]    
    
    #Edge features-------------------------------------
    def start_to_end(self,d_path1,d_path2):
        seg_1 = self.path_to_seg(d_path1)
        seg_2 = self.path_to_seg(d_path2)
        if self.is_close(seg_1.point(0),seg_2.point(0)) or self.is_close(seg_1.point(0),seg_2.point(1)) or self.is_close(seg_1.point(1),seg_2.point(0)) or self.is_close(seg_1.point(1),seg_2.point(1)):
            return True
        else:
            return False
    
    def a_difference(self,angle1,angle2):
        #this is for normalization
        angle_max = 1.5707963268
        #angle must be in radians
        #https://www.nagwa.com/en/explainers/407162748438/
        #90 degrees must be dealt because if m1 * m2 = -1 it gives error.
        m1 = tan(angle1)
        m2 = tan(angle2)
        if self.is_close(1,-(m1*m2)):
            return 1.5707963268 / angle_max
        else:
            tan_alpha = abs((m1-m2)/(1+m1*m2))        
            return (atan(tan_alpha)) / angle_max
    
    def is_close(self,first,second):
        return abs(first - second) < 1e-3
    
    def is_near_90(self,angle):    
        return abs(abs(angle)- pi/2) < 1e-3
    
    def is_near_180(self,angle):
        self.difference= abs(abs(angle)- pi)    
        return (self.difference <= 1e-3) or (abs(self.difference - pi) <= 1e-3)
    
    def positional_offset(self,d_path1,d_path2):
        max_ofset= 70
        seg_1 = self.path_to_seg(d_path1)
        seg_2 = self.path_to_seg(d_path2)
        normalized=(Line(seg_1.point(.5),seg_2.point(.5)).length()) / max_ofset
        offset =(Line(seg_1.point(.5),seg_2.point(.5)).length())
        return normalized,offset
    
    def length_ratio(self,d_path1,d_path2):
        seg_1 = self.path_to_seg(d_path1)
        seg_2 = self.path_to_seg(d_path2)
        return seg_1.length()/(seg_1.length()+seg_2.length())
    
    def is_parallel(self,d_path1,d_path2):

        if self.is_near_180((self.angle_line(d_path1)-self.angle_line(d_path2))):
            return True
        else:
            return False
    
    def is_orthogonal(self,d_path1,d_path2):
        return self.is_near_90((self.angle_line(d_path1)- self.angle_line(d_path2)))
    
    def edge_features(self,d_path1,d_path2):
        angle1 = self.angle_line(d_path1)
        angle2 = self.angle_line(d_path2)
        return [self.positional_offset(d_path1,d_path2)[0],self.length_ratio(d_path1,d_path2),self.is_parallel(d_path1,d_path2),self.is_orthogonal(d_path1,d_path2),self.start_to_end(d_path1,d_path2),self.a_difference(angle1,angle2)]


    #-------------------------------------------------- 
    
    def is_close(self,first,second):
        return abs(first - second) < 1e-3  
    
    def len(self):
        #print(f'len called')
        return len(self.filenames)

    def get(self, idx):
        # print(f'get called')
        data = torch.load(os.path.join(self.processed_dir,f'd_{self.filenames[idx][:-4]}.pt'))   
        return data   

## Datasets (train/val)

## Gat Layer

In [ ]:
#This is  original Gat Layer from PYG

from typing import Optional, Tuple, Union

import torch
import torch.nn.functional as F
from torch import Tensor
from torch.nn import Parameter
from torch_sparse import SparseTensor, set_diag

from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.nn.inits import glorot, zeros
from torch_geometric.typing import Adj, OptTensor, PairTensor
from torch_geometric.utils import add_self_loops, remove_self_loops, softmax


class GATv2Conv(MessagePassing):
    r"""The GATv2 operator from the `"How Attentive are Graph Attention
    Networks?" <https://arxiv.org/abs/2105.14491>`_ paper, which fixes the
    static attention problem of the standard
    :class:`~torch_geometric.conv.GATConv` layer.
    Since the linear layers in the standard GAT are applied right after each
    other, the ranking of attended nodes is unconditioned on the query node.
    In contrast, in :class:`GATv2`, every node can attend to any other node.

    .. math::
        \mathbf{x}^{\prime}_i = \alpha_{i,i}\mathbf{\Theta}\mathbf{x}_{i} +
        \sum_{j \in \mathcal{N}(i)} \alpha_{i,j}\mathbf{\Theta}\mathbf{x}_{j},

    where the attention coefficients :math:`\alpha_{i,j}` are computed as

    .. math::
        \alpha_{i,j} =
        \frac{
        \exp\left(\mathbf{a}^{\top}\mathrm{LeakyReLU}\left(\mathbf{\Theta}
        [\mathbf{x}_i \, \Vert \, \mathbf{x}_j]
        \right)\right)}
        {\sum_{k \in \mathcal{N}(i) \cup \{ i \}}
        \exp\left(\mathbf{a}^{\top}\mathrm{LeakyReLU}\left(\mathbf{\Theta}
        [\mathbf{x}_i \, \Vert \, \mathbf{x}_k]
        \right)\right)}.

    If the graph has multi-dimensional edge features :math:`\mathbf{e}_{i,j}`,
    the attention coefficients :math:`\alpha_{i,j}` are computed as

    .. math::
        \alpha_{i,j} =
        \frac{
        \exp\left(\mathbf{a}^{\top}\mathrm{LeakyReLU}\left(\mathbf{\Theta}
        [\mathbf{x}_i \, \Vert \, \mathbf{x}_j \, \Vert \, \mathbf{e}_{i,j}]
        \right)\right)}
        {\sum_{k \in \mathcal{N}(i) \cup \{ i \}}
        \exp\left(\mathbf{a}^{\top}\mathrm{LeakyReLU}\left(\mathbf{\Theta}
        [\mathbf{x}_i \, \Vert \, \mathbf{x}_k \, \Vert \, \mathbf{e}_{i,k}]
        \right)\right)}.

    Args:
        in_channels (int or tuple): Size of each input sample, or :obj:`-1` to
            derive the size from the first input(s) to the forward method.
            A tuple corresponds to the sizes of source and target
            dimensionalities.
        out_channels (int): Size of each output sample.
        heads (int, optional): Number of multi-head-attentions.
            (default: :obj:`1`)
        concat (bool, optional): If set to :obj:`False`, the multi-head
            attentions are averaged instead of concatenated.
            (default: :obj:`True`)
        negative_slope (float, optional): LeakyReLU angle of the negative
            slope. (default: :obj:`0.2`)
        dropout (float, optional): Dropout probability of the normalized
            attention coefficients which exposes each node to a stochastically
            sampled neighborhood during training. (default: :obj:`0`)
        add_self_loops (bool, optional): If set to :obj:`False`, will not add
            self-loops to the input graph. (default: :obj:`True`)
        edge_dim (int, optional): Edge feature dimensionality (in case
            there are any). (default: :obj:`None`)
        fill_value (float or Tensor or str, optional): The way to generate
            edge features of self-loops (in case :obj:`edge_dim != None`).
            If given as :obj:`float` or :class:`torch.Tensor`, edge features of
            self-loops will be directly given by :obj:`fill_value`.
            If given as :obj:`str`, edge features of self-loops are computed by
            aggregating all features of edges that point to the specific node,
            according to a reduce operation. (:obj:`"add"`, :obj:`"mean"`,
            :obj:`"min"`, :obj:`"max"`, :obj:`"mul"`). (default: :obj:`"mean"`)
        bias (bool, optional): If set to :obj:`False`, the layer will not learn
            an additive bias. (default: :obj:`True`)
        share_weights (bool, optional): If set to :obj:`True`, the same matrix
            will be applied to the source and the target node of every edge.
            (default: :obj:`False`)
        **kwargs (optional): Additional arguments of
            :class:`torch_geometric.nn.conv.MessagePassing`.

    Shapes:
        - **input:**
          node features :math:`(|\mathcal{V}|, F_{in})` or
          :math:`((|\mathcal{V_s}|, F_{s}), (|\mathcal{V_t}|, F_{t}))`
          if bipartite,
          edge indices :math:`(2, |\mathcal{E}|)`,
          edge features :math:`(|\mathcal{E}|, D)` *(optional)*
        - **output:** node features :math:`(|\mathcal{V}|, H * F_{out})` or
          :math:`((|\mathcal{V}_t|, H * F_{out})` if bipartite.
          If :obj:`return_attention_weights=True`, then
          :math:`((|\mathcal{V}|, H * F_{out}),
          ((2, |\mathcal{E}|), (|\mathcal{E}|, H)))`
          or :math:`((|\mathcal{V_t}|, H * F_{out}), ((2, |\mathcal{E}|),
          (|\mathcal{E}|, H)))` if bipartite
    """
    _alpha: OptTensor

    def __init__(
        self,
        in_channels: Union[int, Tuple[int, int]],
        out_channels: int,
        heads: int = 1,
        concat: bool = True,
        negative_slope: float = 0.2,
        dropout: float = 0.0,
        add_self_loops: bool = True,
        edge_dim: Optional[int] = None,
        fill_value: Union[float, Tensor, str] = 'mean',
        bias: bool = True,
        share_weights: bool = False,
        **kwargs,
    ):
        super().__init__(node_dim=0, **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.concat = concat
        self.negative_slope = negative_slope
        self.dropout = dropout
        self.add_self_loops = add_self_loops
        self.edge_dim = edge_dim
        self.fill_value = fill_value
        self.share_weights = share_weights

        if isinstance(in_channels, int):
            self.lin_l = Linear(in_channels, heads * out_channels, bias=bias,
                                weight_initializer='glorot')
            if share_weights:
                self.lin_r = self.lin_l
            else:
                self.lin_r = Linear(in_channels, heads * out_channels,
                                    bias=bias, weight_initializer='glorot')
        else:
            self.lin_l = Linear(in_channels[0], heads * out_channels,
                                bias=bias, weight_initializer='glorot')
            if share_weights:
                self.lin_r = self.lin_l
            else:
                self.lin_r = Linear(in_channels[1], heads * out_channels,
                                    bias=bias, weight_initializer='glorot')

        self.att = Parameter(torch.Tensor(1, heads, out_channels))

        if edge_dim is not None:
            self.lin_edge = Linear(edge_dim, heads * out_channels, bias=False,
                                   weight_initializer='glorot')
        else:
            self.lin_edge = None

        if bias and concat:
            self.bias = Parameter(torch.Tensor(heads * out_channels))
        elif bias and not concat:
            self.bias = Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)

        self._alpha = None

        self.reset_parameters()

    def reset_parameters(self):
        self.lin_l.reset_parameters()
        self.lin_r.reset_parameters()
        if self.lin_edge is not None:
            self.lin_edge.reset_parameters()
        glorot(self.att)
        zeros(self.bias)


    def forward(self, x: Union[Tensor, PairTensor], edge_index: Adj,
                edge_attr: OptTensor = None,
                return_attention_weights: bool = None):
        # type: (Union[Tensor, PairTensor], Tensor, OptTensor, NoneType) -> Tensor  # noqa
        # type: (Union[Tensor, PairTensor], SparseTensor, OptTensor, NoneType) -> Tensor  # noqa
        # type: (Union[Tensor, PairTensor], Tensor, OptTensor, bool) -> Tuple[Tensor, Tuple[Tensor, Tensor]]  # noqa
        # type: (Union[Tensor, PairTensor], SparseTensor, OptTensor, bool) -> Tuple[Tensor, SparseTensor]  # noqa
        r"""
        Args:
            return_attention_weights (bool, optional): If set to :obj:`True`,
                will additionally return the tuple
                :obj:`(edge_index, attention_weights)`, holding the computed
                attention weights for each edge. (default: :obj:`None`)
        """
        H, C = self.heads, self.out_channels

        x_l: OptTensor = None
        x_r: OptTensor = None
        if isinstance(x, Tensor):
            assert x.dim() == 2
            x_l = self.lin_l(x).view(-1, H, C)
            if self.share_weights:
                x_r = x_l
            else:
                x_r = self.lin_r(x).view(-1, H, C)
        else:
            x_l, x_r = x[0], x[1]
            assert x[0].dim() == 2
            x_l = self.lin_l(x_l).view(-1, H, C)
            if x_r is not None:
                x_r = self.lin_r(x_r).view(-1, H, C)

        assert x_l is not None
        assert x_r is not None

        if self.add_self_loops:
            if isinstance(edge_index, Tensor):
                num_nodes = x_l.size(0)
                if x_r is not None:
                    num_nodes = min(num_nodes, x_r.size(0))
                edge_index, edge_attr = remove_self_loops(
                    edge_index, edge_attr)
                edge_index, edge_attr = add_self_loops(
                    edge_index, edge_attr, fill_value=self.fill_value,
                    num_nodes=num_nodes)
            elif isinstance(edge_index, SparseTensor):
                if self.edge_dim is None:
                    edge_index = set_diag(edge_index)
                else:
                    raise NotImplementedError(
                        "The usage of 'edge_attr' and 'add_self_loops' "
                        "simultaneously is currently not yet supported for "
                        "'edge_index' in a 'SparseTensor' form")

        # propagate_type: (x: PairTensor, edge_attr: OptTensor)
        out = self.propagate(edge_index, x=(x_l, x_r), edge_attr=edge_attr,
                             size=None)

        alpha = self._alpha
        self._alpha = None

        if self.concat:
            out = out.view(-1, self.heads * self.out_channels)
        else:
            out = out.mean(dim=1)

        if self.bias is not None:
            out = out + self.bias

        if isinstance(return_attention_weights, bool):
            assert alpha is not None
            if isinstance(edge_index, Tensor):
                return out, (edge_index, alpha)
            elif isinstance(edge_index, SparseTensor):
                return out, edge_index.set_value(alpha, layout='coo')
        else:
            return out


    def message(self, x_j: Tensor, x_i: Tensor, edge_attr: OptTensor,
                index: Tensor, ptr: OptTensor,
                size_i: Optional[int]) -> Tensor:
        x = x_i + x_j
        #Niyazi
        #print('called')
        if edge_attr is not None:
            if edge_attr.dim() == 1:
                edge_attr = edge_attr.view(-1, 1)
            assert self.lin_edge is not None
            edge_attr = self.lin_edge(edge_attr)
            edge_attr = edge_attr.view(-1, self.heads, self.out_channels)
            x = x + edge_attr

        x = F.leaky_relu(x, self.negative_slope)
        alpha = (x * self.att).sum(dim=-1)
        alpha = softmax(alpha, index, ptr, size_i)
        self._alpha = alpha
        alpha = F.dropout(alpha, p=self.dropout, training=self.training)
        mes = x_j * alpha.unsqueeze(-1)
        #Niyazi
        #print(mes.shape)
        return mes

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.in_channels}, '
                f'{self.out_channels}, heads={self.heads})')

## Gat Model

In [ ]:
class GAT(torch.nn.Module):
    """Graph Attention Network"""
    def __init__(self, dim_in, dim_h, dim_out, heads=2):
        super().__init__()        
        self.gat1 = GATv2Conv(dim_in, dim_h, edge_dim=6, heads=7,add_self_loops=False)
        self.gat2 = GATv2Conv(dim_h*heads, dim_h, edge_dim=6, heads=7,add_self_loops=False)
        self.gat3 = GATv2Conv(dim_h*heads, dim_h, edge_dim=6, heads=7,add_self_loops=False)
        self.gat4 = GATv2Conv(dim_h*heads, dim_h, edge_dim=6, heads=7,add_self_loops=False)
        self.gat5 = GATv2Conv(dim_h*heads, dim_h, edge_dim=6, heads=7,add_self_loops=False)
        self.gat6 = GATv2Conv(dim_h*heads, dim_h, edge_dim=6, heads=7,add_self_loops=False)
        self.gat7 = GATv2Conv(dim_h*heads, dim_h, edge_dim=6, heads=7,add_self_loops=False)
        
        self.node_lin_1 = Linear(dim_h*7, dim_out)
        self.node_lin_2 = Linear(dim_out, dim_out)
        self.ats_1 = Linear(505, 256)
        self.ats_2 = Linear(256,216)
        self.ats_3 = Linear(216,128)
        self.ats_4 = Linear(128,64)
        self.ats_5 = Linear(64,32)
        self.ats_6 = Linear(32,16)
        self.ats_7 = Linear(16, 2)
        
        
        self.optimizer = torch.optim.Adam(self.parameters(),
                                          lr=0.0005,betas=(0.95, 0.999),
                                          weight_decay=1e-4)

    def forward(self, x, edge_index,edge_attr):
        #h = F.dropout(x, p=0.6, training=self.training)
        
#         PARAMETERS
#             return_attention_weights (bool, optional) – If set to True,
#             will additionally return the tuple (edge_index, attention_weights),
#             holding the computed attention weights for each edge. (default: None)
        
        #print(f'edge_index:{edge_index.shape},edge_attr : {edge_attr.shape}') 
        h = self.gat1(x, edge_index,edge_attr, return_attention_weights=True)
        ascore1 =  h[1][1].mean(dim=1).unsqueeze(dim=1)
        # h = F.relu(h)
        #print(f'h out:{h[0].shape},h edge index shape: {h[1][0].shape}, h attention_score_shape: {h[1][1].shape}')
        h=h[0]        
        h = F.elu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        
        
        h = self.gat2(h, edge_index,edge_attr,  return_attention_weights=True)
        ascore2 =  h[1][1].mean(dim=1).unsqueeze(dim=1)
        # print(f'out:{h[0].shape},edge index shape: {h[1][0].shape}, attention_score_shape: {h[1][1].shape}')
        h=h[0]
        h = F.elu(h)
        # h = F.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        
        h = self.gat3(h, edge_index,edge_attr, return_attention_weights=True)
        ascore3 =  h[1][1].mean(dim=1).unsqueeze(dim=1)
        # print(f'out:{h[0].shape},edge index shape: {h[1][0].shape}, attention_score_shape: {h[1][1].shape}')
        h=h[0]
        h = F.elu(h)
        # h = F.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        
        h = self.gat4(h, edge_index,edge_attr, return_attention_weights=True)
        ascore4 =  h[1][1].mean(dim=1).unsqueeze(dim=1)
        # print(f'out:{h[0].shape},edge index shape: {h[1][0].shape}, attention_score_shape: {h[1][1].shape}')
        h=h[0]
        h = F.elu(h)
        # h = F.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        
        
        h = self.gat5(h, edge_index,edge_attr, return_attention_weights=True)
        ascore5 =  h[1][1].mean(dim=1).unsqueeze(dim=1)
        # print(f'out:{h[0].shape},edge index shape: {h[1][0].shape}, attention_score_shape: {h[1][1].shape}\n',
        #       f'first vertex att_score i->j, j->i : {h[1][1][0]}, \n first vertex index i->j, j->i : {h[1][0][0][0],h[1][0][1][0]}')

        h=h[0]
        h = F.elu(h)
        # h = F.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        
        h = self.gat6(h, edge_index,edge_attr, return_attention_weights=True)
        # print(f'out:{h[0].shape},edge index shape: {h[1][0].shape}, attention_score_shape: {h[1][1].shape}')
        ascore6 =  h[1][1].mean(dim=1).unsqueeze(dim=1)
        #print(f'ascore:{ascore.shape}')
        h=h[0]
        h = F.elu(h)
        # h = F.relu(h)
        h = F.dropout(h, p=0.5, training=self.training)
        
        
        h = self.gat7(h, edge_index,edge_attr, return_attention_weights=True)
        ascore7 =  h[1][1].mean(dim=1).unsqueeze(dim=1)
        #h[1][1] is attention score, mean is aggregation of 7 attention heads.        
        #print(f'h out:{h[0].shape}, h edge index shape: {h[1][0].shape}, h attention_score_shape: {h[1][1].shape}\n')
        # h[1][0] is edge index
        
        edge_ind = h[1][0]
        # arranging edge indexes [2,num_edge] to [num_edge,2]
        edge_ind = rearrange(edge_ind, 'h w ->  w h')
        
        #this is the output from Gat Layers
        node_embeddings =h[0]
        
        lin_out = self.node_lin_1(node_embeddings)
        lin_out = self.node_lin_2(lin_out)      
        
        

        # print(f'last att_score: {ascore7.shape}')
        # print(f'last att_score: {ascore6.shape}')
        # print(f'last att_score: {ascore5.shape}')
        # print(f'last att_score: {ascore4.shape}')
        # print(f'last att_score: {ascore3.shape}')
        # print(f'last att_score: {ascore2.shape}')
        # print(f'last att_score: {ascore1.shape}')
        att_score_aggregaded = torch.mean(torch.stack((ascore1 , ascore2 , ascore3 , ascore4 , ascore5 , ascore6 ,  ascore7),dim=0),dim=0)
        #print(f'att_score_aggregaded:{att_score_aggregaded.shape}')
        # this is  the concatanation process of (node1,node2,attention score between them)
        #first get the pairs of node embeddings
        pairs = node_embeddings[edge_ind]
        #second put them into a same tensor instead of pairs of two tensors.
        pairs = rearrange(pairs, 'b h w -> b (h w)')
        # third --> (node1,node2,attention)
        edge_emb = torch.cat((pairs,att_score_aggregaded),dim=1)                
        #print(f'att score: {att_scores.shape}',f' node embedding: {h[0].shape}',f'edge index: {h[1][0].shape}' )

        att_out = self.ats_1(edge_emb)
        att_out = self.ats_2(att_out)
        att_out = self.ats_3(att_out)
        att_out = self.ats_4(att_out)
        att_out = self.ats_5(att_out)
        att_out = self.ats_6(att_out)
        att_out = self.ats_7(att_out)
        #print(f'att_out: {att_out[:10]}')  
        #h[0] is node embeddings.
        
        att_out= torch.sigmoid(att_out)
        #att_out= F.softmax(att_out)
        
        
        return F.log_softmax(lin_out, dim=1),att_out

## Accuracy and training

In [ ]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def accuracy2(pred_yy, yy):
    """Calculate accuracy2."""
    #this makes the prediction results(same or not) complement to 2
    sm = torch.nn.Softmax(dim=1)
    soft_pred = sm(pred_yy)
    #this makes wrong predictions zero:
    # tensor([[0.0000, 0.4014],
    #     [0.6602, 0.0000],
    #     [0.5958, 0.0000],
    #     ...,
    #     [0.6334, 0.0000],
    #     [0.6778, 0.0000],
    #     [0.6734, 0.0000]], device='cuda:0')
    out= soft_pred.where(yy == 1 ,yy)
    #print(f'out size: {out.size()}')
    out = out[:,0] + out[:,1]
    #print(f'out size: {out.size()}')
    outTrue = (out > .7)
    #print(f'out True: {outTrue.size()}')
    #print(f'len yy: {len(yy)}')
    #Count Trues
    #print(f'percent:{(torch.count_nonzero(opTrue) / len(yy)).item()}')
    #print(f'nonzero:{torch.count_nonzero(outTrue)}')
    return (torch.count_nonzero(outTrue) / len(yy)).item()

def train(model,train_dl,valid_dl):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss()
    
    optimizer = model.optimizer
    epochs = 100
    model.train()
    for epoch in range(epochs+1):
        # Training
        #print('epoch')
        cumul_loss,tot_loss, tot_acc,tot_acc2,tot_loss1,tot_loss2 = 0.,0.,0.,0.,0.,0.
        #print(len(train_dl))
        
        #thi is number of nodes in every batch
        #datax=[]
        counter = 0.
        for data in train_dl:
            nt = len(train_dl)
            #print(nt)             
                        
            pairs = rearrange(data.edge_index, 'h w -> w h')
            edge_vertex_class = data.y[pairs]
            same_class_or_not = edge_vertex_class[:,0] == edge_vertex_class[:,1]
            
            weight_yy = torch.ones( data.yy.size(),device=device)
            
            weight_yy[:,0][same_class_or_not] = 10            
            criterion2 = torch.nn.BCELoss(weight=weight_yy)
            
            data = data.to(device)
            out1,out2 = model(data.x, data.edge_index,data.edge_attr)
            
            
        
            
            loss1 = criterion(out1, data.y)
            # print(f'same_class_or_not:{same_class_or_not.size()}')
            # print(f'weight_yy:{weight_yy.size()}')
            # print(f'out2:{out2.size()}')
            loss2 = criterion2(out2, data.yy)
            
            # print(f'loss2{loss2:.2f}'
            #       f'target shape{data.yy[:10]}'
            #      )
            loss =loss1 + loss2
            tot_loss1 += loss1
            tot_loss2 += loss2
            tot_loss += loss
            cumul_loss += loss
            acc = accuracy(out1.argmax(dim=1), data.y)
            acc2 = accuracy2(out2, data.yy)
            tot_acc += acc
            tot_acc2 += acc2
            if counter % 5 == 0:
                loss.backward()
                optimizer.step()
                cumul_loss = 0.
                optimizer.zero_grad()
        
        #print(f'number of nodes {max(datax)}')
        
        
        tot_loss = tot_loss/nt
        tot_acc = tot_acc/nt
        tot_acc2 = tot_acc2/nt
        tot_loss1 = tot_loss1/nt
        tot_loss2 = tot_loss2/nt
        
        with torch.no_grad():
            model.eval()
            tot_v_loss, tot_v_acc,tot_v_acc2,tot_v_loss1,tot_v_loss2 = 0.,0.,0.,0.,0.
            #print(len(valid_dl))
            for v_data in valid_dl:
                # Validation
                
                v_data=v_data.to(device)
                pairs = rearrange(v_data.edge_index, 'h w -> w h')
                edge_vertex_class = v_data.y[pairs]
                same_class_or_not = edge_vertex_class[:,0] == edge_vertex_class[:,1]
            
                weight_yy = torch.ones(v_data.yy.size(),device=device)
            
                weight_yy[:,0][same_class_or_not] = 10            
                criterion2 = torch.nn.BCELoss(weight=weight_yy)
                v_out1,v_out2 = model(v_data.x, v_data.edge_index,v_data.edge_attr)
                
                #v_data.yy= torch.abs(torch.cat((data.yy,data.yy-1),dim=1).to(torch.float))
                #print(v_data.yy.shape)
                tot_v_loss1 += criterion(v_out1, v_data.y)
                tot_v_loss2 += criterion2(v_out2, v_data.yy)
                
                tot_v_acc += accuracy(v_out1.argmax(dim=1), v_data.y)
                tot_v_acc2 += accuracy2(v_out2, v_data.yy)
                

            nv = len(valid_dl)
            tot_v_loss1 = tot_v_loss1/nv
            tot_v_loss2 = tot_v_loss2/nv
            tot_v_acc = tot_v_acc/nv
            tot_v_acc2 = tot_v_acc2/nv
        # Print metrics every 10 epochs
        if(epoch % 10 == 0):
            print(f'Epoch {epoch:>3} |\n  '
                  f'Train Loss:  {tot_loss:.2f} | Val Loss:  {tot_v_loss:.2f}\n'                    
                  f'Train Loss1: {tot_loss1:.2f} | Train Loss2: {tot_loss2:.2f}\n '
                  f'Val Loss1: {tot_v_loss1:.2f} | Val Loss2: {tot_v_loss2:.2f}\n '
                  f'Train Acc 1: {tot_acc*100:.2f}% | Train Acc 2: {tot_acc2*100:.2f}%\n '                  
                  f'Val Acc1: {tot_v_acc*100:.2f}% | Val Acc2: {tot_v_acc2*100:.2f}% '
                                    
                  )
          
    return model

## Train

In [ ]:
train_ds = floorPlanCad(path/"dataset_train_full/")
#train_ds = floorPlanCad(path/"_mini_toy_dataset/")

In [ ]:
val_ds = floorPlanCad(path/"dataset_validation_full")
#val_ds = floorPlanCad(path/"_mini_toy_dataset_validation/")

In [ ]:
train_dl = DataLoader(train_ds, batch_size=5, shuffle=True)

In [ ]:
val_dl = DataLoader(val_ds, batch_size=5, shuffle=True)

In [ ]:
gat = GAT(train_ds.num_features, 36, 36,heads=7)

In [ ]:
gat = gat.to('cuda:0')

In [ ]:
#train(gat,train_dl,val_dl)

In [ ]:
#torch.save(gat.state_dict(), path/"models/untrained_test_full_dataset.pt")

In [ ]:
gat.load_state_dict(torch.load(path/"models/84_72_100_7_layers_att_sigmoid_beta_95_1e-4-full_dataset.pt"))

<All keys matched successfully>

In [ ]:
num_to_name = dict()

In [ ]:
for i in range(36):
    num_to_name[str(i+1)]= str(i+1)

In [ ]:
num_to_name['1'] = 'wall'
num_to_name['3'] = 'single door'
num_to_name['4'] = 'double door'
num_to_name['5'] = 'sliding door'
num_to_name['9'] = 'window-9'
num_to_name['13'] = 'sofa'
num_to_name['14'] = 'bed'
num_to_name['15'] = 'chair'
num_to_name['16'] = 'table'
num_to_name['17'] = 'tv cabinet'
num_to_name['18'] = 'wardrobe'
num_to_name['19'] = 'cabinet'
num_to_name['20'] = 'refrigerator'
num_to_name['22'] = 'gas stove'
num_to_name['23'] = 'sink'
num_to_name['29'] = 'toilet'
num_to_name['31'] = 'elevator'



In [ ]:
#

## Inference

In [ ]:
def infer(model,v_data):      
    with torch.no_grad():
        v_data.edge_index,v_data.edge_attr = remove_self_loops(v_data.edge_index,v_data.edge_attr)
        model.eval()
        v_data=v_data.to('cuda:0')
        v_out1,v_out2 = model(v_data.x, v_data.edge_index,v_data.edge_attr)
        print(
            f'data: {v_data} |\n |'
            f'class_shape: {v_out1.shape} | '
            f'instance_shape: {v_out2.shape}')
    return v_out1,v_out2

In [ ]:
#data = torch.load(path/"_mini_toy_dataset_validation/processed/d_0969-0027.pt")
data = torch.load(path/"dataset_validation_full/processed/d_0001-0072.pt")
#data = torch.load(path/"dataset_validation_full/processed/d_0000-0003.pt")
data

Data(x=[1212, 4], edge_index=[2, 21890], edge_attr=[21890, 6], y=[1212], yy=[21890, 2])

In [ ]:
v1,v2 = infer(gat,data)
#v1.shape,v2.shape

data: Data(x=[1212, 4], edge_index=[2, 21890], edge_attr=[21890, 6], y=[1212], yy=[21890, 2]) |
 |class_shape: torch.Size([1212, 36]) | instance_shape: torch.Size([21890, 2])


# Check Accuracy2

In [ ]:
v2.size()

torch.Size([21890, 2])

In [ ]:
data.yy

tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]], device='cuda:0')

In [ ]:
v2

tensor([[0.6188, 0.5262],
        [0.6036, 0.5090],
        [0.6188, 0.5262],
        ...,
        [0.9531, 0.1122],
        [0.9527, 0.1144],
        [0.9509, 0.1188]], device='cuda:0')

In [ ]:
m = torch.nn.Softmax(dim=1)
input = v2
output = m(data.yy)
output

tensor([[0.7311, 0.2689],
        [0.7311, 0.2689],
        [0.7311, 0.2689],
        ...,
        [0.7311, 0.2689],
        [0.7311, 0.2689],
        [0.7311, 0.2689]], device='cuda:0')

In [ ]:
m = torch.nn.Softmax(dim=1)
input = v2
output = m(input)
output

tensor([[0.5231, 0.4769],
        [0.5236, 0.4764],
        [0.5231, 0.4769],
        ...,
        [0.6987, 0.3013],
        [0.6981, 0.3019],
        [0.6968, 0.3032]], device='cuda:0')

In [ ]:
#vv = v2[:10,:].where(yy10 == 1 ,yy10)

In [ ]:
op= output.where(data.yy == 1 ,data.yy)
op

tensor([[0.5231, 0.0000],
        [0.5236, 0.0000],
        [0.5231, 0.0000],
        ...,
        [0.6987, 0.0000],
        [0.6981, 0.0000],
        [0.6968, 0.0000]], device='cuda:0')

In [ ]:
op = op[:,0] + op[:,1]
op

tensor([0.5231, 0.5236, 0.5231,  ..., 0.6987, 0.6981, 0.6968], device='cuda:0')

I think it is working

In [ ]:
opTrue = (op > .5)

print(opTrue)

torch.count_nonzero(opTrue)

tensor([True, True, True,  ..., True, True, True], device='cuda:0')


tensor(16001, device='cuda:0')

In [ ]:
9537/12542

0.760405039068729

In [ ]:
visualisation = path/"dataset_validation_full/raw/0001-0072.svg"
#visualisation = path/"_mini_toy_dataset_validation/raw/0969-0027.svg"

In [ ]:
paths, attributes = svg2paths(visualisation)

## **This part made for checking if the model could predict the real connection between members of a instance.**

#### **Clean attributes and paths from drawing not from the results**

In [ ]:
count = 0
atts = []
pths = []
#available class list in the drawing
drw_sem_id_list = []
for att in attributes:    
    if ('semantic-id' and 'instance-id') in att:
        idx = attributes.index(att)
        atts.append(attributes[idx])
        pths.append(paths[idx])
        if int(att['semantic-id']) not in drw_sem_id_list:
            drw_sem_id_list.append(int(att['semantic-id']))
        
len(atts),len(pths), sorted(drw_sem_id_list)

(1212,
 1212,
 [1, 3, 9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 26, 29, 33])

## **Vertex Prediction**

### **This is how many class in this particular prediction**

In [ ]:
prediction_sem_id_list = set(v1.argmax(dim=1).tolist())

In [ ]:
len(prediction_sem_id_list), prediction_sem_id_list

(22,
 {1,
  3,
  4,
  9,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  26,
  27,
  29,
  30,
  32,
  33})

In [ ]:
preds = v1.argmax(dim=1).tolist()
len(preds)

1212

In [ ]:
#vertex class predictions
vertex_dict = {}
for i in  prediction_sem_id_list:
    vertex_dict[str(i)]= []
    
for idx, pred in enumerate(preds):
    temp_list = vertex_dict[str(pred)]
    temp_list.append(idx)
    vertex_dict[str(pred)] = temp_list
        
#print(vertex_dict['15'])

In [ ]:
vertex_dict

{'1': [53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  166,
  167,
  168,
  169,
  170,
  171,
  172,
  173,
  174,
  175,
  176,
  177,
  178,
  179,
  180,
  181,
  182,
  183,
  184,
  185,
  186,
  187,
  188,
  189,
  190,
  191,
  192,
  193,
  194,
  197,
  200,
  218,
  234,
  235,
  246,
  287,
  288,
  289,
  290,
  291,
  345,

In [ ]:
#all edges in pairs
pairs = rearrange(data.edge_index, 'h w -> w h')
len(pairs), pairs.size()

(21890, torch.Size([21890, 2]))

In [ ]:
#data.yy[:10]  same_class_or_not

this is for loss weight matrix

In [ ]:
edge_vertex_class = data.y[pairs]
same_class_or_not = edge_vertex_class[:,0] == edge_vertex_class[:,1]


In [ ]:
edge_vertex_class.size()

torch.Size([21890, 2])

In [ ]:
v2,v2.mean(dim=0)

(tensor([[0.6188, 0.5262],
         [0.6036, 0.5090],
         [0.6188, 0.5262],
         ...,
         [0.9531, 0.1122],
         [0.9527, 0.1144],
         [0.9509, 0.1188]], device='cuda:0'),
 tensor([0.8510, 0.2633], device='cuda:0'))

In [ ]:
#possibility_tensor_bool = (v2[:,0]>0.8) & (v2[:,1]<0.5)
possibility_tensor_bool = (v2[:,0]>v2[:,1]) & (v2[:,1]<0.5)
possibility_tensor_bool

tensor([False, False, False,  ...,  True,  True,  True], device='cuda:0')

In [ ]:
pairs = pairs[possibility_tensor_bool]

In [ ]:
# pairs in list format
pairs = pairs.tolist()
pairs[0],len(pairs)

([1, 4], 16716)

In [ ]:
#edge dictionary and idx's 
edge_dict = {}
for i in  prediction_sem_id_list:
    edge_dict[str(i)]= []
edge_dict_idx = {}
for i in  prediction_sem_id_list:
    edge_dict_idx[str(i)]= []

In [ ]:
#extracting pairs for each classes
for k,v in vertex_dict.items():
    for idx,i in  enumerate(pairs):
        if (i[0] in v) and (i[1] in v):
            edge_dict[k].append(i)
            edge_dict_idx[k].append(idx)

In [ ]:
edge_dict_idx

{'1': [1357,
  1358,
  1359,
  1360,
  1361,
  1362,
  1367,
  1368,
  1369,
  1370,
  1371,
  1372,
  1381,
  1382,
  1389,
  1390,
  1391,
  1392,
  1393,
  1394,
  1395,
  1396,
  1397,
  1398,
  1399,
  1400,
  1401,
  1402,
  1403,
  1404,
  1405,
  1408,
  1409,
  1410,
  1419,
  1420,
  1421,
  1422,
  1423,
  1431,
  1432,
  1433,
  1434,
  1435,
  1446,
  1447,
  1448,
  1449,
  1450,
  1451,
  1456,
  1457,
  1458,
  1459,
  1460,
  1461,
  1470,
  1471,
  1472,
  1473,
  1474,
  1475,
  1476,
  1477,
  1478,
  1479,
  1480,
  1481,
  1482,
  1483,
  1484,
  1485,
  1486,
  1487,
  1488,
  1489,
  1495,
  1496,
  1497,
  1498,
  1499,
  1504,
  1505,
  1506,
  1507,
  1508,
  1509,
  1510,
  1511,
  1512,
  1518,
  1519,
  1520,
  1521,
  1522,
  1523,
  1524,
  1525,
  1526,
  1532,
  1533,
  1534,
  1535,
  1536,
  1537,
  1538,
  1539,
  1540,
  1541,
  1542,
  1543,
  1544,
  1545,
  1546,
  1547,
  1548,
  1549,
  1551,
  1552,
  1553,
  1554,
  1555,
  1556,
  1557,
  1

In [ ]:
# removing vertexes that don't have a connection in that class
# this can be considered as verification of previous step.
# no need to make this if vertex dict not necessary anymore which
# is optional. it can be done during the loop
for k,v in edge_dict.items():
    nodes = set()
    nodes = nodes.union(set(item[0] for item in v))
    nodes = nodes.union(set(item[1] for item in v))
    for i in vertex_dict[k]:
        if i not in nodes:
            vertex_dict[k].remove(i)  
    

In [ ]:
# links = [
#     (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (1, 7), (7, 9), (7, 8), (8, 9), # Component 1
#     (10, 11), (11, 12), (10, 12) # Component 2
# ]

# # Create list of all nodes
# nodes = set()
# nodes = nodes.union(set(item[0] for item in links))
# nodes = nodes.union(set(item[1] for item in links))

# # Create a data structure in the form of  from -> List<to>
# # {"1":[2,7],"2":[3],"3":[4],"4":[5],"5":[6],"7":[9,8],"8":[9],"10":[11,12],"11":[12]}
# links_by_nodes = {}
# for x, y in links:
#     if x in links_by_nodes:
#         links_by_nodes[x].append(y)
#     else:
#         links_by_nodes[x] = [y]

def find_connected_components(all_nodes, links_by_nodes):
    connected_components = []
    all_component_nodes = set()
    for node in all_nodes:
        if node in all_component_nodes:
            continue
        component = set()
        dfs(links_by_nodes, component, all_component_nodes, node)
        connected_components.append(component)
    return connected_components

def dfs(links_by_nodes, component, all_component_nodes, node):
    component.add(node)
    all_component_nodes.add(node)

    linked_nodes = links_by_nodes.get(node)
    if linked_nodes is None:
        return
    for linked_node in linked_nodes:
        if linked_node in all_component_nodes:
            continue
        dfs(links_by_nodes, component, all_component_nodes, linked_node)
        


In [ ]:
edge_dict

{'1': [[111, 112],
  [111, 114],
  [111, 119],
  [111, 122],
  [111, 123],
  [111, 124],
  [112, 111],
  [112, 115],
  [112, 119],
  [112, 122],
  [112, 123],
  [112, 115],
  [112, 355],
  [112, 356],
  [114, 111],
  [114, 115],
  [114, 122],
  [114, 124],
  [114, 115],
  [114, 180],
  [114, 182],
  [115, 112],
  [115, 114],
  [115, 123],
  [115, 124],
  [115, 180],
  [115, 182],
  [116, 117],
  [116, 120],
  [116, 200],
  [116, 218],
  [117, 116],
  [117, 200],
  [117, 218],
  [119, 111],
  [119, 112],
  [119, 122],
  [119, 123],
  [119, 200],
  [120, 116],
  [120, 126],
  [120, 128],
  [120, 126],
  [120, 128],
  [122, 111],
  [122, 112],
  [122, 114],
  [122, 119],
  [122, 123],
  [122, 124],
  [123, 111],
  [123, 112],
  [123, 115],
  [123, 119],
  [123, 122],
  [123, 115],
  [123, 355],
  [123, 356],
  [124, 111],
  [124, 114],
  [124, 115],
  [124, 122],
  [124, 115],
  [124, 180],
  [124, 182],
  [115, 112],
  [115, 114],
  [115, 123],
  [115, 124],
  [115, 180],
  [115, 182],
 

In [ ]:
instance_dict = {}
for k,v in edge_dict.items():
    links_by_nodes = {}
    for x, y in edge_dict[k]:
        if x in links_by_nodes:
            links_by_nodes[x].append(y)
        else:
            links_by_nodes[x] = [y]
    instance_dict[k]=find_connected_components(vertex_dict[k], links_by_nodes)
    

In [ ]:
the_list = []
for semantic_key,instance_index_lists  in instance_dict.items():
    for instance_indexes in instance_index_lists:            
        temp_list=[]
        the_dict = {}
        for idx in instance_indexes:            
            temp_list.append(pths[idx])
        the_dict[semantic_key] = temp_list
        the_list.append(the_dict)

In [ ]:
def boundingbox(xmin,xmax,ymin,ymax):
    spath=svg_path(Line(start=(xmin+ymin), end=(xmax+ymin)),
             Line(start=(xmax+ymin), end=(xmax+ymax)),
             Line(start=(xmax+ymax), end=(xmin+ymax)),
             Line(start=(xmin+ymax), end=(xmin+ymin)))
    return spath
# def min_max_bbox(instance_paths):    
#     real_list = []
#     imag_list = []
#     for path in instance_paths:
#         # print(path)
#         if isinstance(path[0],Line):
#             real_list.extend((path[0].start.real,path[0].end.real))
#             imag_list.extend((path[0].start.imag,path[0].end.imag))
#         elif isinstance(path[0],Arc):
#             if path[0].start.real < path[0].end.real:
#                 center_real = (path[0].start.real + path[0].radius.real) 
#                 center_imag = path[0].start.imag                
#                 bottom_real = center_real
#                 bottom_imag = center_imag - path[0].radius.imag
#                 #print(f'bottom: {center_imag,bottom_imag}')
#                 real_list.extend((path[0].start.real, path[0].end.real, bottom_real))
#                 imag_list.extend((path[0].start.imag, path[0].end.imag, bottom_imag))
#             if path[0].start.real > path[0].end.real:
#                 center_real = (path[0].start.real - path[0].radius.real)  
#                 center_imag = path[0].start.imag
#                 top_real = center_real 
#                 top_imag = center_imag + path[0].radius.imag
#                 #print(f'top: {center_imag,bottom_imag}')
#                 real_list.extend((path[0].start.real,path[0].end.real, top_real))
#                 imag_list.extend((path[0].start.imag,path[0].end.imag, top_imag))
def min_max_bbox(instance_paths):    
    real_list = []
    imag_list = []
    for path in instance_paths:  
        
        real_list.extend((path[0].start.real,path[0].end.real,path[0].point(0.5).real))
        imag_list.extend((path[0].start.imag,path[0].end.imag,path[0].point(0.5).imag))
         
    xmin = min(real_list)
    xmax = max(real_list)
    ymin = min(imag_list)*1j
    ymax = max(imag_list)*1j
    return xmin,xmax,ymin,ymax
           

In [ ]:
instance_borders =[]
instance_text = []
# this is where the text goes
instance_path = []
for i in the_list:
    if  list(i.items())[0][0] != '1':
        ins_lines = list(i.items())[0][1]
        ins_name = list(i.items())[0][0]
        mmx= min_max_bbox(ins_lines)
        bbx_borders = boundingbox(*mmx)    
        instance_borders.append(bbx_borders)
        instance_text.append(ins_name)
        instance_path.append(bbx_borders[0])

In [ ]:
tmp_list = list()
for i in instance_text:
    tmp_list.append(num_to_name[i])

In [ ]:
instance_text=tmp_list

In [ ]:
border_atts = []
for i in instance_borders:
    border_atts.append({'stroke': 'rgb(255,0,0)','fill': 'none','stroke-width': '0.2'})

In [ ]:
wsvg(pths+instance_borders,attributes= atts+border_atts,font_size=2, text=instance_text, text_path=instance_path, filename='_0969-0027.svg') 

if semantic segmentation underperforms then it means :
- if it skips some connection that is necessary. it is destructive because  it could lead us to find many more instances than actual.(false negatives)
- if it makes false positives for a class I think it is less destructive because we can filter them out during instance extraction
if instance extraction underperforms then it means:
- if it creates false positives then it leads to find much less instances
- if it creates false negatives